We use Mistral-7*8B model to evaluate similarity of any combination of two questions.

### Calculation

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "/fl/model/mixtral/mixtral-instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16, attn_implementation="flash_attention_2")

/opt/miniconda3/envs/xtuner/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████████████████| 19/19 [00:23<00:00,  1.23s/it]


In [2]:
def mixtral_inf(text, token_len=512):
    inputs = tokenizer(text, return_tensors="pt").to("cuda") 

    outputs = model.generate(**inputs, max_new_tokens=token_len, temperature=0)
    return tokenizer.decode(outputs[0])

In [3]:
def format(sent1, sent2):
    sent = f"""<s> [INST] **Objective**: Assess the similarity of two questions not just based on their word content and relatedness, but primarily based on their underlying purpose or intent. This assessment should distinguish between questions that are about opposite aspects (even if related) and those that are different in wording but similar in the information they seek. Mark it from 0-100. 0 means absolutely no same intention, 100 means completely same intention. You don't need to explain your choice, just output a score in the format of similarity score: <the score>.
    
    **Procedure**:
    1. Read and Understand the Questions:
        - Carefully read the given questions.
        - Understand the core purpose behind each question.
    2. Identify Key Aspects:
        - Determine what each question is specifically asking for.
        - Identify if the questions are seeking positive, negative, or neutral aspects of a subject.
    3. Analyze Semantic Differences:
        - Compare the intent behind the questions.
        - Look for opposing themes such as benefits vs drawbacks, advantages vs disadvantages, etc.
    4. Similarity Scoring:
        - Assign a high similarity score if both questions are asking about the same aspect of a subject, even if the wording is different.
        - Assign a low similarity score if the questions are asking about opposite aspects, even if they are related or use similar words.
    
    **Examples**:
    Question 1: "What are the pros of using zeolites?"
    Question 2: "What are the drawbacks of using zeolites?"
    Analysis: Though both questions are about the zeolites, one asks about pros (positive aspects) while the other asks about drawbacks (negative aspects).
    Similarity Score: 0
    
    Question 1: "Give some examples of using zeolites as catalysts."
    Question 2: "What are potential applications of using zeolites as catalysts?"
    Analysis: Both questions seek information on how zeolites are used as catalysts, with slightly different wording.
    Similarity Score: 100
    
    **Notes:**
    Pay attention to nuanced language that may indicate different intentions even in closely related topics.
    Remember, a high degree of relatedness does not necessarily imply a high degree of similarity in intent.
    
    **Questions**
    Question1: {sent1}
    Question2: {sent2} [/INST]"""

    return sent
# print(sent)

In [4]:
import json
def read(file_path):
    with open(file_path, 'r') as file:
        datas = json.load(file) 
    return datas

# define the evaluted file using file_path variable
file_path = '/fl/data/acl/acl_eval_final_vicuna16k_all.json'
datas = read(file_path)
print(len(datas))

105


In [5]:
from tqdm.notebook import tqdm
import re

def cal_mistral(data):
    questions = [qa['Q'].replace('Q: ', '') for qa in data['Q&A']]
    answer_dic = {}
    for i in range(len(questions)):
        # print(i)
        for j in range(i):
            if i != j:
                q1 = data['Q&A'][i]['Q']
                q2 = data['Q&A'][j]['Q']
                sent = format(q1, q2)
                answer_dic[str((i,j))] = mixtral_inf(sent, token_len=128).split('[/INST]')[1]
    return process(answer_dic)


# process generated answer to seperate similarity score and reason
def process(answer_dic):
    process_answer_dic = {}
    for i in answer_dic:
        answer = answer_dic[i]
        try:
            part = answer.split('\n\n')
            score = part[0].split(': ')[1]
            match = re.search(r'-?\d+(\.\d+)?', score)
            if match:
                score = match.group()
            else:
                score = 0
                print(i)
            reason = part[1]
            process_answer_dic[i] = {'score': int(score)/100, 'reason': reason}
        except:
            print(i)

    return process_answer_dic

In [6]:
# generate results and record to file for each ten generated results
res = []
res_part = []

with open(f'q_sim_output.json', "w") as file:
    pass

for i in range(len(datas)):
    print(i)
    data = datas[i]
    process_answer_dic = cal_mistral(data)
    res.append(process_answer_dic)
    res_part.append(process_answer_dic)
    if i%10 == 0:
        with open(f'q_sim_output.json', "a") as file:
            file.writelines([json.dumps(item) + "\n" for item in res_part])
        res_part = []

/opt/miniconda3/envs/xtuner/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

103


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

104


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

### Visualization

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import ast

from pathlib import Path
# pth = 'q_sim/mistral'
# Path(f'{pth}/q_sim_all').mkdir(parents=True)
# Path(f'{pth}/q_sim(0.5)').mkdir(parents=True)
# Path(f'{pth}/fig').mkdir(parents=True)

def mistral_vis(data_idx):
    pth = 'q_sim/mistral'
    fn = datas[data_idx]['doi'].replace('/', '-')
    num_qa = datas[data_idx]['num_Q&A']

    process_answer_dic = res[data_idx]
    q_all_pth = f'{pth}/q_sim_all/{fn}.json'
    with open(q_all_pth, 'w') as json_file:
        json.dump(process_answer_dic, json_file, indent=4)

    matrix = [[0 for _ in range(num_qa)] for _ in range(num_qa)]
    for i in process_answer_dic:
        i = ast.literal_eval(i)
        matrix[i[0]][i[1]] = process_answer_dic[str(i)]['score']
    matrix = np.array(matrix)
    
    indices = np.argwhere(matrix > 0.5)
    
    # Visualization
    fig, ax = plt.subplots(figsize=(8, 8))
    mask = np.triu(np.ones_like(matrix, dtype=bool))
    
    cax = ax.matshow(np.ma.masked_where(mask, matrix), cmap='coolwarm', vmin=0, vmax=1)  # Red-Blue colormap
    plt.colorbar(cax, fraction=0.045, pad=0.04) 
    # plt.colorbar(cax)
    
    # plt.title("Question Similarity")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    for (i, j), val in np.ndenumerate(matrix):
        if i>j:
            ax.text(j, i, f"{val:.2f}", ha='center', va='center')
    
    # Setting labels for x and y axis
    ax.set_xticks(np.arange(num_qa))
    ax.set_yticks(np.arange(num_qa))
    ax.set_xticklabels(np.arange(num_qa))
    ax.set_yticklabels(np.arange(num_qa))
    
    # Highlight indices greater than 0.5
    for (x, y) in indices:
        plt.scatter(y, x, color='red', s=50)  # Swap x and y for correct positioning
    
    # plt.show()
    plt.savefig(f'{pth}/fig/{fn}.png')
    plt.close(fig)
    
    # print("Indices of elements greater than 0.5:", indices, len(indices))
    idx_dic = {}
    for (x,y) in indices:
        q1 = datas[data_idx]['Q&A'][x]['Q']
        q2 = datas[data_idx]['Q&A'][y]['Q']
        idx_dic[str((x,y))] = {'score': matrix[x][y], 'questions': f'Question1: {q1}\nQuestion2: {q2}', 'reason': process_answer_dic[str((x,y))]['reason']}
    
    select_tokens_pth = f'{pth}/q_sim(0.5)/{fn}.json'
    with open(select_tokens_pth, 'w') as json_file:
        json.dump(idx_dic, json_file, indent=4)

In [23]:
for i in range(len(datas)):
    mistral_vis(i)

### Count

In [9]:
# Occurence of each similarity score
count_dic = {}
pth = 'q_sim/mistral'

for data_idx in range(len(datas[:100])):
    fn = datas[data_idx]['doi'].replace('/', '-')
    
    q_all_pth = f'{pth}/q_sim_all/{fn}.json'
    file = read(q_all_pth)

    for i in file:
        score = file[i]['score']
        if score not in count_dic:
            count_dic[score] = 1
        else:
            count_dic[score] += 1
sorted(count_dic.items(), key=lambda x:x[0])

[(0.0, 536),
 (0.1, 121),
 (0.2, 2395),
 (0.3, 55),
 (0.4, 2),
 (0.5, 420),
 (0.6, 114),
 (0.7, 478),
 (0.8, 302),
 (0.9, 31),
 (1.0, 46)]

In [10]:
# Average similarity score for all questions
sum([i[0]*i[1] for i in count_dic.items()])/sum(count_dic.values())

0.3193111111111111